In [ ]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup

# 使用者資訊模擬
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

# 查詢網站
url = requests.get('https://www.104.com.tw/jobs/search/?ro=1&isnew=30&keyword=%E8%B3%87%E6%96%99%E7%A7%91%E5%AD%B8&area=6001001000%2C6001002000%2C6001008000&order=12&asc=0&page=1&mode=l' , headers = headers).url
driver = webdriver.Chrome()
driver.get(url)

# 自動下滑功能
for i in range(20): 
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.6)
    
# 自動加載
k = 1
while k != 0:
    try:
        driver.find_elements_by_class_name("js-more-page",)[-1].click() 
        print('Click 手動載入，' + '載入第' + str(15 + k) + '頁')
        k = k+1
        time.sleep(1) 
    except:
        k = 0
        print('No more Job')

# BeautifulSoup解析資料
soup = BeautifulSoup(driver.page_source, 'html.parser')
List = soup.findAll('a',{'class':'js-job-link'})
print('共有 ' + str(len(List)) + ' 筆資料')

Name = soup.select('.job-mode__jobname')
Job_Name = [e.text for e in Name]
del Job_Name[0]

Company = soup.select('.job-mode__company')
Company_Name = [e.text for e in Company]
del Company_Name[0]

Link =  soup.select('.js-job-link')
df_link=[]
for i in Link:
    df_link.append(i.get('href'))
      
df = pd.DataFrame({'Job Name' : Job_Name, "Company Name" : Company_Name,"Link" : df_link})
df['Job Name'].replace('\n','',regex=True,inplace=True)
df['Company Name'].replace('\n','',regex=True,inplace=True)
df['Link'].replace('//','',regex=True,inplace=True)
df.reset_index(drop=True)
Select_df=df[df['Job Name'].str.contains("資料科學")]

writer = pd.ExcelWriter('output.xlsx',index=False)
Select_df.to_excel(writer,'Sheet1',index=False)
df.to_excel(writer,'Sheet2')
writer.save()